In [ ]:
import duckdb
import pandas as pd
%load_ext sql

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False


In [ ]:
%sql duckdb:///:default:
# %sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [ ]:
# Obtain credentials to access Minio store
from stelar.auth import Realm
from stelar.store import S3Store

In [ ]:
stelarstaging2_realm = Realm("https://authst.vsamtuc.top/", "stelarstaging2")
stelarstaging2_realm.well_known()

In [ ]:
miniost_secret = "AmNzFfupuvN4OVs9loWl5F3714LNDGvz"
miniost_client = stelarstaging2_realm.client("miniost", miniost_secret)


In [ ]:
alice_token = miniost_client.token("alice", "stelar_alice")
alice_token

In [ ]:
test_minio_tenant = S3Store("https://miniost.vsamtuc.top")
alice_jwt_token = alice_token['access_token']
alice_role = test_minio_tenant.assume_role_with_web_identity(alice_jwt_token)
alice_role

In [ ]:
alice_credentials = alice_role['AssumeRoleWithWebIdentityResponse']['AssumeRoleWithWebIdentityResult']['Credentials']
alice_access_key_id = alice_credentials['AccessKeyId']
alice_secret_access_key = alice_credentials['SecretAccessKey']
alice_session_token = alice_credentials['SessionToken']


In [ ]:
%%sql
SET s3_region= 'us-east-1';
SET s3_endpoint= 'miniost.vsamtuc.top';
SET s3_url_style= 'path';
SET s3_access_key_id= '{{alice_access_key_id}}';
SET s3_secret_access_key='{{alice_secret_access_key}}';
SET s3_session_token= '{{alice_session_token}}';


In [ ]:
%%sql 
SELECT * 
FROM  's3://bucket001/myslides/psl-state-transitions.parquet'
LIMIT 5